3655-9102-2UVYT3

In [1]:
const λ = 1.0
const Z = 2.0
const _ordenTaylor = 20
const ɛ = 1.0e-20

1.0e-20

In [2]:
using Plots, TaylorSeries, LaTeXStrings
pyplot(legend = :false, grid = :false)

Plots.PyPlotBackend()

In [32]:
# Returns stepsize of the integration and a vector with the updated values of the dependent
# variables
function taylorStepper{T<:Real}( jetEqs::Function, vec0::Array{T,1} )
    
    n = length( vec0 )

    vec0T = Array(Taylor1{T},n)
    @simd for i in eachindex(vec0)
        @inbounds vec0T[i] = Taylor1([vec0[i]], _ordenTaylor)
    end

    # Jets
    vec1T = jetEqs( vec0 )
    
    # Step-size
    hh = Inf
    for i in eachindex(vec1T)
        @inbounds h1 = stepsize( vec1T[i], ɛ )
        hh = min( hh, h1 )
    end
    
    # Values at t0+h
    @simd for i in eachindex(vec0)
        @inbounds vec0[i] = evaluate( vec1T[i], hh )
    end
    
    return hh, vec0
end

# Returns the maximum step size from epsilon and the last two coefficients of the x-Taylor series 
function stepsize{T<:Real}(x::Taylor1{T}, epsilon::Float64)
    ord = x.order
    h = Inf
    for k in [ord-1, ord]
        kinv = 1.0/k
        aux = abs( x.coeffs[k+1] )
        h = min(h, (epsilon/aux)^kinv)
    end
    return h
end


function jetundriven{T<:Real}( vec::Array{T,1} )

    tT = Taylor1(vec[1], _ordenTaylor)
    Q1T = Taylor1(vec[2], _ordenTaylor)
    P1T = Taylor1(vec[3], _ordenTaylor)
    Q2T = Taylor1(vec[4], _ordenTaylor)
    P2T = Taylor1(vec[5], _ordenTaylor)
    
    # Auxiliary quantities
    sQ1 = zeros( T, _ordenTaylor+1 )
    sP1 = zeros( T, _ordenTaylor+1 )
    cQ1 = zeros( T, _ordenTaylor+1 )
    cQ2 = zeros( T, _ordenTaylor+1 )
    sQ2 = zeros( T, _ordenTaylor+1 )
    sP2 = zeros( T, _ordenTaylor+1 )
    f = zeros( T, _ordenTaylor+1 )
    absf = zeros( T, _ordenTaylor+1 )
    absf3 = zeros( T, _ordenTaylor+1 )
    
   
    
    
    s11P1 = zeros( T, _ordenTaylor+1 )
    s1P1 = zeros( T, _ordenTaylor+1 )
    P1d1 = zeros( T, _ordenTaylor+1 )
    s21P1 = zeros( T, _ordenTaylor+1 )
    div2 = zeros( T, _ordenTaylor+1 )
    s2P1 = zeros( T, _ordenTaylor+1 )
        
    s11P2 = zeros( T, _ordenTaylor+1 )
    s1P2 = zeros( T, _ordenTaylor+1 )
    P2d1 = zeros( T, _ordenTaylor+1 )
    s21P2 = zeros( T, _ordenTaylor+1 )
    s2P2 = zeros( T, _ordenTaylor+1 )
    
    
    dotQ1 = zeros( T, _ordenTaylor+1 )
    dotP1 = zeros( T, _ordenTaylor+1 )
    dotQ2 = zeros( T, _ordenTaylor+1 )
    dotP2 = zeros( T, _ordenTaylor+1 )
    dott = zeros( T, _ordenTaylor+1 )
    
    
    
    # Now the implementation
    for k = 0:_ordenTaylor-1
        knext = k+1
        # The right-hand size of the eqs of motion
        # This is more adpated for this problem, and avoids many superflous operations
           
        sQ1[knext] = TaylorSeries.squareHomogCoef(k, Q1T.coeffs)
        sP1[knext] = TaylorSeries.squareHomogCoef(k, P1T.coeffs)
        sQ2[knext] = TaylorSeries.squareHomogCoef(k, Q2T.coeffs)
        sP2[knext] = TaylorSeries.squareHomogCoef(k, P2T.coeffs)
        cQ1[knext] = TaylorSeries.powHomogCoef(k, Q1T.coeffs, 3, cQ1, 0)
        cQ2[knext] = TaylorSeries.powHomogCoef(k, Q2T.coeffs, 3, cQ2, 0)
        
               
        
        
        f[knext] = sQ1[knext] - sQ2[knext]  
        absf[knext] = sign(f[1])*f[knext]
        absf3[knext] = TaylorSeries.powHomogCoef(k, f, 3, absf3, 0)
        
       
        
        
        P1d1[knext] = TaylorSeries.divHomogCoef(k, sQ2, absf, P1d1, 0)
        s11P1[knext] = -0.125*sP2[knext] + Z - P1d1[knext] - sQ2[knext]
        s1P1[knext] = 2.0*TaylorSeries.mulHomogCoef(k, Q1T.coeffs, s11P1)
        s21P1[knext] = 2.0*TaylorSeries.mulHomogCoef(k, cQ1, sQ2)
        div2[knext] = TaylorSeries.divHomogCoef(k, f, absf3, div2, 0)
        s2P1[knext] = TaylorSeries.mulHomogCoef(k, s21P1, div2)
        
        #@show -0.125*sP2[knext] + Z - P1d1[knext] - sQ2[knext], -0.125*sP2[knext]
        
        
        P2d1[knext] = TaylorSeries.divHomogCoef(k, sQ1, absf, P2d1, 0)
        s11P2[knext] = -0.125*sP1[knext] + Z - P2d1[knext] - sQ1[knext]
        s1P2[knext] = 2.0*TaylorSeries.mulHomogCoef(k, Q2T.coeffs, s11P2)
        s21P2[knext] = 2.0*TaylorSeries.mulHomogCoef(k, cQ2, sQ1)
        s2P2[knext] = TaylorSeries.mulHomogCoef(k, s21P2, div2)
    
        
        dotQ1[knext] = 0.25*TaylorSeries.mulHomogCoef(k,sQ2, P1T.coeffs)
        dotP1[knext] = s1P1[knext] + s2P1[knext]
        dotQ2[knext] = 0.25*TaylorSeries.mulHomogCoef(k,sQ1, P2T.coeffs)        
        dotP2[knext] = s1P2[knext] - s2P2[knext]             
        dott[knext] = TaylorSeries.mulHomogCoef(k, sQ1, sQ2)
        
        #@show "That's ok"
        Q1T.coeffs[knext + 1] = dotQ1[knext]/knext        
        P1T.coeffs[knext + 1] = dotP1[knext]/knext        
        Q2T.coeffs[knext + 1] = dotQ2[knext]/knext        
        P2T.coeffs[knext + 1] = dotP2[knext]/knext        
        tT.coeffs[knext + 1] = dott[knext]/knext
    end
    
    return Taylor1[tT, Q1T, P1T, Q2T, P2T]
end
#Creamos condiciones iniciales
function condini(x10::Float64, px10::Float64)
    @assert x10 != 0
    Q1 = sqrt(x10)
    Q2 = 0.0
    P1 = px10*sqrt(x10)    
    P2 = sqrt(8Z)    
    return [0, Q1, P1, Q2, P2]
end

condini (generic function with 1 method)

Aquí implementaremos el hamiltoniano regularizado, con $E = -1$, debe cumplirse que $H = 0 \ \forall t$, primero, tomaremos una función auxiliar:

In [30]:
v = condini(1.2, 1.0)

5-element Array{Float64,1}:
 0.0    
 1.09545
 1.09545
 0.0    
 4.0    

In [33]:
jetundriven(v)

5-element Array{TaylorSeries.Taylor1,1}:
                                0.5759999999999995 t³ + NaN t⁴ + NaN t⁵ + NaN t⁶ + NaN t⁷ + NaN t⁸ + NaN t⁹ + NaN t¹⁰ + NaN t¹¹ + NaN t¹² + NaN t¹³ + NaN t¹⁴ + NaN t¹⁵ + NaN t¹⁶ + NaN t¹⁷ + NaN t¹⁸ + NaN t¹⁹ + NaN t²⁰ + 𝒪(t²¹)
           1.0954451150103321 + 0.1314534138012398 t³ + NaN t⁴ + NaN t⁵ + NaN t⁶ + NaN t⁷ + NaN t⁸ + NaN t⁹ + NaN t¹⁰ + NaN t¹¹ + NaN t¹² + NaN t¹³ + NaN t¹⁴ + NaN t¹⁵ + NaN t¹⁶ + NaN t¹⁷ + NaN t¹⁸ + NaN t¹⁹ + NaN t²⁰ + 𝒪(t²¹)
  1.0954451150103321 + 2.1908902300206643 t² + NaN t³ + NaN t⁴ + NaN t⁵ + NaN t⁶ + NaN t⁷ + NaN t⁸ + NaN t⁹ + NaN t¹⁰ + NaN t¹¹ + NaN t¹² + NaN t¹³ + NaN t¹⁴ + NaN t¹⁵ + NaN t¹⁶ + NaN t¹⁷ + NaN t¹⁸ + NaN t¹⁹ + NaN t²⁰ + 𝒪(t²¹)
                        1.1999999999999997 t + NaN t³ + NaN t⁴ + NaN t⁵ + NaN t⁶ + NaN t⁷ + NaN t⁸ + NaN t⁹ + NaN t¹⁰ + NaN t¹¹ + NaN t¹² + NaN t¹³ + NaN t¹⁴ + NaN t¹⁵ + NaN t¹⁶ + NaN t¹⁷ + NaN t¹⁸ + NaN t¹⁹ + NaN t²⁰ + 𝒪(t²¹)
                                4.0 + NaN t² + NaN 

In [35]:
taylorStepper(jetundriven, v)

(Inf,[NaN,NaN,NaN,NaN,NaN])

In [4]:
ks(Q) = Q^2

ks (generic function with 1 method)

In [5]:
function regHam(t, Q₁, P₁, Q₂, P₂)
    #Cantidades auxiliares
    P₁² = P₁^2
    P₂² = P₂^2
    Q₁² = Q₁^2
    Q₂² = Q₂^2
    nf = sqrt(((Q₁)^2 - (Q₂)^2)^2)
    
    H = 0.125*(P₁²*Q₂² + P₂²*Q₁²) - Z*(Q₁² + Q₂²) + Q₁²*Q₂²*(1.0 + 1.0/nf)
    return H
end


regHam(v) = regHam(v...)

regHam (generic function with 2 methods)

In [19]:
v = condini(5.4,1.)

5-element Array{Float64,1}:
 0.0    
 2.32379
 2.32379
 0.0    
 4.0    

In [20]:
taylorStepper(jetundriven, v)

(Inf,[NaN,NaN,NaN,NaN,NaN])

In [9]:
var1 = set_variables("t q1 p1 q2 p2", order = 1)
function errHam(N::Int)
    dQ1 = zeros(N)
    dQ2 = zeros(N)
    dP1 = zeros(N)
    dP2 = zeros(N)
    als = typeof(zeros(5))[]
    for j in 1:N
        al = 10*rand(5)
        alt = al + var1
        ene = regHam(alt)
        ts, meq = taylorStepper(jetundriven, al)
        
        dQ1[j] = abs(meq[2] - ene.coeffs[2].coeffs[3])
        dQ2[j] = abs(meq[4] - ene.coeffs[2].coeffs[5])
        dP1[j] = abs(meq[3] + ene.coeffs[2].coeffs[2])
        dP2[j] = abs(meq[5] + ene.coeffs[2].coeffs[4])
        push!(als, al)
    end
    return dQ1, dQ2, dP1, dP2, als        
end

errHam (generic function with 1 method)

In [10]:
errHam(10)

([46.5359,13.5552,0.744034,146.088,16.214,55.89,40.7633,2.80455,7.37523,0.59959],[4.16954,75.6414,0.878443,33.2017,22.9804,18.4236,3.63582,8.90917,0.805768,2.98843],[262.693,5208.5,5.1045,1717.79,631.365,991.894,118.134,7.32157,45.473,42.4028],[120.336,6603.99,7.98252,1252.72,325.707,613.003,27.3124,26.4472,13.1255,1.34925],Array{Float64,1}[[11.4889,3.06019,3.50692,6.5514,4.61206],[5.32036,6.17387,2.74348,6.04154,7.85875],[6.39448,0.126673,4.37563,0.884261,7.1037],[17.4232,8.32848,4.71801,9.76642,0.586942],[5.25906,4.75516,-0.234752,7.97699,4.59257],[12.2752,6.03319,1.26888,9.03341,1.8326],[8.86701,1.46515,3.20326,6.44402,7.57479],[8.74196,3.33863,1.72569,1.26033,3.24526],[4.45407,1.58412,2.33737,3.51435,4.76783],[2.88089,1.39748,-0.175408,3.95787,1.85513]])

Implementaremos también la función para regresar las velocidades al espacio fase de la configuración original... 

In [ ]:
function ksv(Q, P)
    v₁ = (Q*P)
    return v₁
end
ks(Q) = Q^2

In [ ]:
function undrivenHelium(τ, q)
    t, Q₁, P₁, Q₂, P₂ = q
    
    t = Q₁^2*Q₂^2
    q₁ = 0.25*Q₂^2*P₁
    p₁ = 2Q₁*(-0.125P₂^2 + Z  - Q₂^2 - Q₂^2/abs(Q₁^2 - Q₂^2)) + 2*Q₁^3*Q₂^2*((Q₁^2 - Q₂^2)/abs(Q₁^2 - Q₂^2)^3)
    q₂ = 0.25*Q₁^2*P₂
    p₂ = 2Q₂*(-0.125P₁^2 + Z  - Q₁^2 - Q₁^2/abs(Q₁^2 - Q₂^2)) - 2*Q₁^2*Q₂^3*((Q₁^2 - Q₂^2)/abs(Q₁^2 - Q₂^2)^3)

    return [t, q₁, p₁, q₂, p₂] 

end

In [ ]:
taylo

condini (generic function with 1 method)

In [ ]:
function undrivenIntegration(x10::Float64, px10::Float64, time_max::Float64, jetEqs::Function )
    
    Q₁, Q₂, P₁, P₂ = condini(x10, px10)
    τ₀ = 0.0
    t0 = 0.0
    
    ene0 = regHam(Q₁, Q₂, P₁, P₂)
        
    Q1v, Q2v, P1v, P2v = Float64[], Float64[], Float64[], Float64[]
    tv, τv = Float64[], Float64[]
    DeneV= Float64[]
    push!(tv, t0)
    push!(τv, τ₀)
    push!(Q1v, Q₁)
    push!(P1v, P₁)
    push!(Q2v, Q₂)
    push!(P2v, P₂)
       
    push!(DeneV, ene0)
    
    # This is the main loop; we include a minimum step size for security
    dτ = 1.0e-7
    while τ₀ < time_max && dτ>1.0e-8
        #Preparamos el vector para integrar
        temp1 = Float64[t0, Q₁ , Q₂, P₁, P₂]
        # Here we integrate
        dτ, (t, Q1, P1, Q2, P2) = taylorStepper( jetEqs, temp1);
        
        τ₀ += dτ
        push!(tv, t)
        push!(τv, τ₀)
        push!(Q1v, Q1)        
        push!(P1v, P1)        
        push!(Q2v, Q2)        
        push!(P2v, P2)
        eneEnd = regHam(Q1, P1, Q2, P2)
        
        dEne = abs(eneEnd - ene0)
        
        push!(DeneV, dEne)
        t0, Q₁ , Q₂, P₁, P₂ = copy(t), copy(Q1), copy(Q2), copy(P1), copy(P2)
    end

    return τv, tv, Q1v, P1v, Q2v, P2v, DeneV
end

Para la configuración de $Zee$ en 1D necesitamos que $y_1, y_2 = 0$; $p_{y_1}, p_{y_2} = 0$, además, necesitamos la condición de que la partícula 2 colisione con el núcleo, por lo que $x_2 = 0$ sobre las coordenadas $(\mathbf{P}, \mathbf{Q})$ implican que:

\begin{align}
    x_1 = x_{1_0} &\Leftrightarrow Q_{11}^2 - Q_{12}^2 = x_{1_0}\\
    y_1 = 0 &\Leftrightarrow 2Q_{11}Q_{12} = 0\\
    x_2 = 0 &\Leftrightarrow Q_{21}^2 - Q_{22}^2 = x_{2_0}\\
    y_2 = 0 &\Leftrightarrow 2Q_{21}Q_{22} = 0 \\
    p_{x_1} = px_{1_0} &\Leftrightarrow Q_{11}P_{11} - Q_{12}P_{12} = px_{1_0}\\
    p_{y_1} = 0 &\Leftrightarrow Q_{12}P_{11} + Q_{11}P_{12} = 0\\
    p_{x_2} = px_{2_0} &\Leftrightarrow Q_{21}P_{21} - Q_{22}P_{22} = px_{2_0}\\
    p_{y_2} = 0 &\Leftrightarrow Q_{22}P_{21} + Q_{21}P_{22} = 0
\end{align}


A partir de esto, podemos concluir que:

\begin{align}
    Q_{11} &= \sqrt{x_{1_0}}\\
    Q_{12} &= 0\\
    Q_{21} &= 0\\
    Q_{22} &= 0\\
    P_{11} &= \frac{px_{1_0}}{Q_{11}}\\
    P_{12} &= 0\\
    P_{22} &= 0
\end{align}

Para las condiciones sobre los momento $P_{21} = 0$ de la partícula 2, debemos considerar la condición sobre la energía del sistema (considerando el hamiltoniano regularizado $H^{SC}$), pues recordemos que se debe cumplir que $H = 0$. Usando las condiciones ya encontradas se halla que debe cumplirse que $|P_2|^2 = Z = 2$, entonces:

\begin{align}
    Q_{11} &= \sqrt{x_{1_0}}\\
    Q_{12} &= 0\\
    Q_{21} &= 0\\
    Q_{22} &= 0\\
    P_{11} &= p_{x_{1}}\sqrt{x_{1_0}}\\
    P_{12} &= 0\\
    P_{21} &= 2\sqrt{2Z}\\
    P_{22} &= 0
\end{align}


In [ ]:
@time tv, τv, Q1v, P1v, Q2v, P2v, DeneV = undrivenIntegration(5.1, 1.4, 2.5, jetundriven);

In [ ]:
Q1v

In [ ]:
DeneV

In [ ]:
plot(eachindex(DeneV), DeneV)

In [ ]:
#Con esto regreso al espacio de configuración inicial
x1 = map(ks, Q1v)
vx1 = Float64[ksv(Q1v[j], P1v[j]) for j in eachindex(Q1v)];

In [ ]:
plot(x1,vx1)